In [1]:
from model import run_model
from sklearn.model_selection import train_test_split
from pathlib import Path

In [ ]:
TARGET = 'sex'

# Find all CSV files in subfolders of data/
data_dir = Path('data/fitness')
csv_files = [f for f in data_dir.rglob('*.csv') if f.parent != data_dir]

# Dictionary to store results
results = {}

# Iterate over each CSV file
for csv_file in csv_files:
    filename = str(csv_file)
    
    try:
        ROC_AUCs = run_model(filename, TARGET)
        results[filename] = ROC_AUCs
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        results[filename] = None

for filename, roc_aucs in results.items():
    print(f"{filename}: {roc_aucs}")

[PosixPath('data/fitness/correlated/minority_10pct.csv'), PosixPath('data/fitness/correlated/minority_30pct.csv'), PosixPath('data/fitness/correlated/minority_3pct.csv'), PosixPath('data/fitness/correlated/minority_1pct.csv'), PosixPath('data/fitness/smote/minority_10pct_smote.csv'), PosixPath('data/fitness/smote/minority_1pct_smote.csv'), PosixPath('data/fitness/smote/minority_30pct_smote.csv'), PosixPath('data/fitness/smote/minority_3pct_smote.csv'), PosixPath('data/fitness/independent/minority_10pct.csv'), PosixPath('data/fitness/independent/minority_10pct_deduplicated.csv'), PosixPath('data/fitness/independent/minority_30pct.csv'), PosixPath('data/fitness/independent/minority_3pct.csv'), PosixPath('data/fitness/independent/minority_1pct.csv')]
Error processing data/fitness/correlated/minority_10pct.csv: "['sex'] not found in axis"
Error processing data/fitness/correlated/minority_30pct.csv: "['sex'] not found in axis"
Error processing data/fitness/correlated/minority_3pct.csv: "['s